In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
np.set_printoptions(4, suppress=True)
torch.set_printoptions(4, sci_mode=False)
import sys
sys.path.append('../')
from ttgo import TTGO
import tt_utils
from utils import test_ttgo
from ur10_kinematics import Ur10Kinematics
from manipulator_utils import dist_orientation_fixed
device = 'cpu'#torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ", device)

device:  cpu


In [2]:
use_trained_model = False

In [3]:
if use_trained_model:
    trained_model = torch.load('ur10_ik.pickle')
    b_goal,b_orient = trained_model['b']
    d0_theta = trained_model['d0_theta']
    dh_x = trained_model['dh_x']
    domain = trained_model['domain']
    Rd_0 = trained_model['Rd_0']
    test_task = trained_model['test_task']

In [4]:
# Setup the robot and the environment

ur10 = Ur10Kinematics(device=device)
n_joints= ur10.n_joints
# Desired orientation (fixed orientation)
Rd_0 = torch.eye(3).to(device)
# Rd_0[1:,1:]*=-1
def cost_all(x): # For inverse kinematics
    x = x.to(device)
    batch_size = x.shape[0]
    goal_loc = x[:,:3]
    q = x[:,3:]  # batch x joint angles
    _, end_loc, end_R = ur10.forward_kin(q) # batch x joint x keys x positions
    # cost on error in end-effector position
    d_goal = torch.linalg.norm(end_loc-goal_loc, dim=1)
    # cost on error in end-effector orientation
    d_orient = dist_orientation_fixed(Rd_0,end_R,device=device)
    c_total = 0.5*(d_goal/b_goal + d_orient/b_orient)
    c_return = torch.cat((c_total.view(-1,1), d_goal.view(-1,1), d_orient.view(-1,1)),dim=-1)
    return c_return

def cost(x):
    return cost_all(x)[:,0]


def pdf(x):
    x = x.to(device)
    pdf_ = torch.exp(-cost(x)**2) 
    return pdf_

#####################################################################


In [5]:
Rd_0

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [6]:
if not use_trained_model:
    # Define the domain
    d0_theta = 60;
    dh_x = 0.01
    d_type = 'uniform'
    b_goal = 0.05; b_orient = 0.2;

    d_theta_all = [d0_theta]*n_joints
    d_theta = [int(d_theta_all[joint]) for joint in range(n_joints)]
    if d_type == 'uniform':
        domain_decision = [torch.linspace(0.5*ur10.theta_min[i],0.5*ur10.theta_max[i],d_theta[i]).to(device) for i in range(n_joints)]
    else: # logarithmic scaling
        domain_decision = [exp_space(0.5*ur10.theta_min[i].to('cpu'),0.5*ur10.theta_max[i].to('cpu'),d_theta[i]).to(device) for i in range(n_joints)]

    # Find the work-space
    n_test = 1000
    test_theta = torch.zeros(n_test,n_joints).to(device)
    for i in range(n_joints):
        unif = torch.distributions.uniform.Uniform(low = domain_decision[i][0],high=domain_decision[i][-1])
        test_theta[:,i]= torch.tensor([unif.sample() for i in range(n_test)]).to(device)
    _, test_xpos, _ = ur10.forward_kin(test_theta)
    x_min,_ = torch.min(test_xpos, dim=0)
    x_max,_ = torch.max(test_xpos,dim=0)
    x_min[-1] = 0.1
    idx_select = test_xpos[:,-1]>x_min[-1]
    test_task = test_xpos[idx_select,:]

    # discretize the domain
    domain_task = [torch.linspace(x_min[i], x_max[i], int((x_max[i]-x_min[i])/dh_x)).to(device) for i in range(3)]
    domain = domain_task + domain_decision
    print("Discretization: ",[len(x) for x in domain])
    tt_model = tt_utils.cross_approximate(fcn=pdf,  domain=[x.to(device) for x in domain], 
                            rmax=200, nswp=20, eps=1e-3, verbose=True, 
    # Refine the discretization and interpolate the model
    scale_factor = 10
    site_list = torch.arange(len(domain))#len(domain_task)+torch.arange(len(domain_decision))
    domain_new = tt_utils.refine_domain(domain=domain, 
                                        site_list=site_list,
                                        scale_factor=scale_factor, device=device)
    tt_model_new = tt_utils.refine_model(tt_model=tt_model.to(device), 
                                        site_list=site_list,
                                        scale_factor=scale_factor, device=device)                        kickrank=5, device=device)
    
    
    ttgo = TTGO(tt_model=tt_model_new, domain=domain_new, cost=cost,device=device)

    
    
    # Save
    file_name = 'ur10_ik.pickle'
    torch.save({
    'tt_model':ttgo.tt_model,
    'b': (b_goal,b_orient),
    'd0_theta':d0_theta,
    'dh_x': dh_x,
    'domain': domain_new,
    'Rd_0':Rd_0,
    'test_task':test_task
    }, file_name)


else:
    tt_model = trained_model['tt_model'].to(device)
    ttgo = TTGO(tt_model = tt_model.to(device), domain=domain,cost=cost)


Discretization:  [249, 251, 132, 60, 60, 60, 60, 60, 60]
cross device is cpu
Cross-approximation over a 9D domain containing 3.84906e+17 grid points:
Note: The algorithm converges as the ratio tt-new-norm/tt-old-norm settles to 1. For TTGO, the convergence is not important, just keep iterating as long as the ratio > 1
iter: 0  | tt-new-norm/tt-old-norm: 0.000e+00 | time:   0.0554 | largest rank:   1
iter: 1  | tt-new-norm/tt-old-norm: 1.975e+13 | time:   0.1947 | largest rank:   4
iter: 2  | tt-new-norm/tt-old-norm: 1.211e+00 | time:   0.4516 | largest rank:   7
iter: 3  | tt-new-norm/tt-old-norm: 1.375e+00 | time:   0.9230 | largest rank:  10
iter: 4  | tt-new-norm/tt-old-norm: 1.222e+00 | time:   1.5941 | largest rank:  13
iter: 5  | tt-new-norm/tt-old-norm: 1.114e+00 | time:   2.5614 | largest rank:  16
iter: 6  | tt-new-norm/tt-old-norm: 1.010e+00 | time:   3.9184 | largest rank:  19
iter: 7  | tt-new-norm/tt-old-norm: 1.015e+00 | time:   5.7780 | largest rank:  22
iter: 8  | tt-ne

In [7]:
.

SyntaxError: invalid syntax (<ipython-input-7-a5d5b61aa8a6>, line 1)

## Visualization

In [8]:
import pybullet_data
from panda_visualization_utils import *
import pybullet as p
from functools import partial
# import the environment (SDF and for graphics visualization in pybullet)
import sys

import sys
DATA_PATH = './data'
robot_urdf = DATA_PATH + '/urdf/ur10/ur10.urdf'


physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

p.resetSimulation()

# for i in range(8):
#     print(i, p.getJointInfo(robot_id, i)[1])
    
robot_id = p.loadURDF(robot_urdf)

dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(1,7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
set_q_std = partial(set_q,robot_id, pb_joint_indices)

plane_id = p.loadURDF('plane.urdf')
p.resetBasePositionAndOrientation(plane_id, (0,0,0.), (0,0,0,1))

#for visualizing the desired target
_,_,ball_id = create_primitives(radius=0.05)

ee_pb_id = 7


In [ ]:
# # Prepare for the task
# sites_task = list(range(3))
# ttgo.set_sites(sites_task)

In [12]:
s = np.random.randint(0,test_task.shape[0]-1)
sample_xe = test_task[s] #
print(sample_xe)


n_solutions= 10
n_samples_tt = 100*n_solutions
n_samples_rand= 1*n_samples_tt

alpha=0.9; 

t1 = time.time()
samples_tt, samples_idx = ttgo.sample_tt(n_samples=n_samples_tt, x_task=sample_xe.reshape(1,-1),alpha=alpha)
state_k_tt = ttgo.choose_top_k_sample(samples_tt,n_solutions)[0]

#Optimize
state_k_tt_opt = 1*state_k_tt
for i, state in enumerate(state_k_tt):
    state_k_tt_opt[i,:],results= ttgo.optimize(state,bound=True)
t2 = time.time()
             
c_tt =  cost_all(state_k_tt_opt)
print("Cost-mean-tt:",torch.mean(c_tt,dim=0))



tensor([-0.27, -0.22,  0.86])
Cost-mean-tt: tensor([0.00, 0.00, 0.00])


In [13]:
x_target = sample_xe[:3].cpu().numpy()
joint_angles_k = state_k_tt[:,3:].cpu().numpy() 
joint_angles_k_opt = state_k_tt_opt[:,3:].cpu().numpy() 



In [14]:
# _,_,test_sphere = create_primitives(p.GEOM_SPHERE, radius = 0.02)
# p.resetBasePositionAndOrientation(test_sphere, (0,0,1.), (0,0,0,1))
_ , _,sphere_id = create_primitives(p.GEOM_SPHERE, radius = 0.02)
pos = x_target[:]

p.resetBasePositionAndOrientation(sphere_id, pos, (0,0,0,1))


k = joint_angles_k.shape[0]
dt = 1
dT = 2
for i in range(2*k):
    set_q_std(joint_angles_k[i%k])
    time.sleep(dt)
    set_q_std(joint_angles_k_opt[i%k])
    time.sleep(2*dt)
    

KeyboardInterrupt: 